# Data Preparation.

In [211]:
# Carregando as bibliotecas
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [212]:
# Leitura dos dados
df = pd.read_csv('../Data/teste_indicium_precificacao.csv')
df.head()

,id,nome,host_id,host_name,bairro_group,bairro,latitude,longitude,room_type,price,minimo_noites,numero_de_reviews,ultima_review,reviews_por_mes,calculado_host_listings_count,disponibilidade_365
0,2595,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,1,45,2019-05-21,0.38,2,355
1,3647,THE VILLAGE OF HARLEM....NEW YORK !,4632,Elisabeth,Manhattan,Harlem,40.80902,-73.94190,Private room,150,3,0,NaN,NaN,1,365
2,3831,Cozy Entire Floor of Brownstone,4869,LisaRoxanne,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,89,1,270,2019-07-05,4.64,1,194
3,5022,Entire Apt: Spacious Studio/Loft by central park,7192,Laura,Manhattan,East Harlem,40.79851,-73.94399,Entire home/apt,80,10,9,2018-11-19,0.10,1,0
4,5099,Large Cozy 1 BR Apartment In Midtown East,7322,Chris,Manhattan,Murray Hill,40.74767,-73.97500,Entire home/apt,200,3,74,2019-06-22,0.59,1,129


### ultima_review

Transformei os dados em formato datetime e após isso efetuei o agrupamento por semestre usando o método `pd.cut()`, assim transformando em variável categórica chamada `ultima_review_semestre`. 

In [164]:
# Transformo a variável em formato data
df['ultima_review'] = pd.to_datetime(df['ultima_review'])

In [165]:
# Obtenho o mínimo e máximo
min_date = df['ultima_review'].min()
max_date = df['ultima_review'].max()

print(F'min_date: {min_date}\nmax_date: {max_date}')

min_date: 2011-03-28 00:00:00
max_date: 2019-07-08 00:00:00


In [166]:
# Adiciono a quantidade de mes ideal para que o semestres fiquem completos
min_date -= pd.DateOffset(months=2)
max_date += pd.DateOffset(months=5)
print(F'min_date: {min_date}\nmax_date: {max_date}')

min_date: 2011-01-28 00:00:00
max_date: 2019-12-08 00:00:00


In [167]:
# Crio o range de datas separados em 6 meses
bins = pd.date_range(start=min_date, end=max_date, freq='6ME')
bins

DatetimeIndex(['2011-01-31', '2011-07-31', '2012-01-31', '2012-07-31',
               '2013-01-31', '2013-07-31', '2014-01-31', '2014-07-31',
               '2015-01-31', '2015-07-31', '2016-01-31', '2016-07-31',
               '2017-01-31', '2017-07-31', '2018-01-31', '2018-07-31',
               '2019-01-31', '2019-07-31'],
              dtype='datetime64[ns]', freq='6ME')

In [168]:
# Faço um loop for com os bins para criar uma lista com as labels do meu agrupamento.
labels = []
for b in bins:
    if b.month == 1:
        labels.append(f'{b.year}-1-semestre')
    else:
        labels.append(f'{b.year}-2-semestre')

''' 
Apago o ultimo elemento da lista justamente para minhas labels tenha 1 elemento a menos
seguindo a sintaxe do pd.cut()   
'''    
labels.pop()
labels

['2011-1-semestre',
 '2011-2-semestre',
 '2012-1-semestre',
 '2012-2-semestre',
 '2013-1-semestre',
 '2013-2-semestre',
 '2014-1-semestre',
 '2014-2-semestre',
 '2015-1-semestre',
 '2015-2-semestre',
 '2016-1-semestre',
 '2016-2-semestre',
 '2017-1-semestre',
 '2017-2-semestre',
 '2018-1-semestre',
 '2018-2-semestre',
 '2019-1-semestre']

In [169]:
# Criei a coluna de agrupamento semestral
df['ultima_review_semestre'] = pd.cut(df['ultima_review'], bins=bins, labels=labels)

# Exibindo os dados
df[['ultima_review_semestre', 'ultima_review']].head()

,ultima_review_semestre,ultima_review
0,2019-1-semestre,2019-05-21
1,NaN,NaT
2,2019-1-semestre,2019-07-05
3,2018-2-semestre,2018-11-19
4,2019-1-semestre,2019-06-22


In [170]:
# Mostrando informações descritivas
df[['ultima_review_semestre','ultima_review']].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48894 entries, 0 to 48893
Data columns (total 2 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   ultima_review_semestre  38842 non-null  category      
 1   ultima_review           38842 non-null  datetime64[ns]
dtypes: category(1), datetime64[ns](1)
memory usage: 430.5 KB


### minimo_noites

Utilizei o método `pd.cut` para criar a categoria `minimo_noites_categorico` com os seguintes agrupamentos:

- Entre_1_a_3_Dias
- Entre_3_a_7_Dias
- Entre_1 e 2 Semanas
- Entre_2_Semanas_e_1_Mes
- Entre_1_Mes_e_2_Meses
- Entre_2_Meses_e_6_Meses
- Entre_6_Meses_e_1_Ano
- Mais_de_1_Ano

In [171]:
# Criei as bins representando o intervalo 
bins = [0, 3, 7, 14, 30, 60, 180, 365 ,float('inf')]

# Criei as labels para cada intervalo
labels = ['Entre_1_a_3_Dias',
          'Entre_3_a_7_Dias', 
          'Entre_1_e_2_Semanas', 
          'Entre_2_Semanas_e_1_Mes',   
          'Entre_1_Mes_e_2_Meses',
          'Entre_1_Meses_e_6_Meses',
          'Entre_6_Meses_e_1_Ano',
          'Mais_de_1_Ano']

# Criei a coluna de agrupamento minimo_noites de forma categórica
df['minimo_noites_categorico'] = pd.cut(df['minimo_noites'], bins=bins,labels=labels).astype('object')

In [172]:
df[['minimo_noites_categorico', 'minimo_noites']].head() 

,minimo_noites_categorico,minimo_noites
0,Entre_1_a_3_Dias,1
1,Entre_1_a_3_Dias,3
2,Entre_1_a_3_Dias,1
3,Entre_1_e_2_Semanas,10
4,Entre_1_a_3_Dias,3


In [173]:
df[['minimo_noites_categorico', 'minimo_noites']].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48894 entries, 0 to 48893
Data columns (total 2 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   minimo_noites_categorico  48894 non-null  object
 1   minimo_noites             48894 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 764.1+ KB


### Disponibilidade 365

Utilizei o método `pd.cut` para criar a categoria `disponibilidade_365_categorico` com os seguintes agrupamentos:

- 0_Dias
- Entre_1_a_3_Dias
- Entre_3_a_7_Dias
- Entre_1 e 2 Semanas
- Entre_2_Semanas_e_1_Mes
- Entre_1_Mes_e_2_Meses
- Entre_2_Meses_e_6_Meses
- Entre_6_Meses_e_1_Ano
- 1_Ano

In [174]:
# Criei as bins representando o intervalo 
bins = [-1,0, 3, 7, 14, 30, 60, 180, 364 , float('inf')]

# Criei as labels para cada intervalo
labels = ['0_Dias',
          'Entre_1_a_3_Dias',
          'Entre_3_a_7_Dias', 
          'Entre_1 e 2 Semanas', 
          'Entre_2_Semanas_e_1_Mes',   
          'Entre_1_Mes_e_2_Meses',
          'Entre_2_Meses_e_6_Meses',
          'Entre_6_Meses_e_1_Ano',
          '1_Ano']

# Criei a coluna de agrupamento disponibilidade_365 de forma categórica
df['disponibilidade_365_categorico'] = pd.cut(df['disponibilidade_365'], bins=bins, labels=labels)

In [175]:
df[['disponibilidade_365_categorico','disponibilidade_365']].head()

,disponibilidade_365_categorico,disponibilidade_365
0,Entre_6_Meses_e_1_Ano,355
1,1_Ano,365
2,Entre_6_Meses_e_1_Ano,194
3,0_Dias,0
4,Entre_2_Meses_e_6_Meses,129


In [176]:
df[['disponibilidade_365_categorico','disponibilidade_365']].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48894 entries, 0 to 48893
Data columns (total 2 columns):
 #   Column                          Non-Null Count  Dtype   
---  ------                          --------------  -----   
 0   disponibilidade_365_categorico  48894 non-null  category
 1   disponibilidade_365             48894 non-null  int64   
dtypes: category(1), int64(1)
memory usage: 430.2 KB


### Preenchendo valores ausentes

Verifiquei se existem valores presentes quando não existem reviews, após isso confirmei que não possui valores e assim efetuei o preenchimento dos valores ausentes nas colunas `ultima_review` e `semestre` como "anuncio_sem_review".
A coluna `reviews_por_mes` preenchi com valores "0" para esses registros.

Criei uma coluna chamada `valor_prenchido` do tipo booleana indicando que os valores foram preenchidos, justamente para usar na etapa da modelagem e ter a possibilidade penalizar esses valores preenchidos, fazendo assim o modelo atribuir menos importancia a esses valores.

In [177]:
df[['numero_de_reviews', 'ultima_review', 'reviews_por_mes']].query('numero_de_reviews == 0').info()

<class 'pandas.core.frame.DataFrame'>
Index: 10052 entries, 1 to 48893
Data columns (total 3 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   numero_de_reviews  10052 non-null  int64         
 1   ultima_review      0 non-null      datetime64[ns]
 2   reviews_por_mes    0 non-null      float64       
dtypes: datetime64[ns](1), float64(1), int64(1)
memory usage: 314.1 KB


In [178]:
df['ultima_review_semestre'] = df['ultima_review_semestre'].cat.add_categories('anuncio_sem_review')
df[['ultima_review','ultima_review_semestre']] = df[['ultima_review','ultima_review_semestre']].fillna('anuncio_sem_review')

In [179]:
len(df.query('reviews_por_mes == 0'))

0

In [180]:
df['reviews_por_mes'] = df['reviews_por_mes'].fillna(0)
len(df.query('reviews_por_mes == 0'))

10052

In [181]:
df['valor_preenchido'] = df['reviews_por_mes'].apply(lambda x: True if x==0 else False)

In [182]:
df[['numero_de_reviews', 'ultima_review', 'reviews_por_mes','ultima_review_semestre','valor_preenchido']].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48894 entries, 0 to 48893
Data columns (total 5 columns):
 #   Column                  Non-Null Count  Dtype   
---  ------                  --------------  -----   
 0   numero_de_reviews       48894 non-null  int64   
 1   ultima_review           48894 non-null  object  
 2   reviews_por_mes         48894 non-null  float64 
 3   ultima_review_semestre  48894 non-null  category
 4   valor_preenchido        48894 non-null  bool    
dtypes: bool(1), category(1), float64(1), int64(1), object(1)
memory usage: 1.2+ MB


### Price

Como discutido na etapa data understanding, a variável price possui 11 valores atribuidos como "0", representando possivelmente um erro de coleta/atribuição.

Efetuei a remoção dos mesmos.

In [183]:
len(df[df['price'] == 0])

11

In [184]:
df = df[df['price'] != 0]

### Corrigindo a contagem de calculado_host_listings_count

Na analise exploratória, identifiquei que alguns `host_id` possuia uma quantidade diferente da mostrada em `calculado_host_listings_count`.

Desta forma efetuei a correção desses valores.


In [185]:
# Exemplo do problema identificado
df.query(f'calculado_host_listings_count == 2').value_counts('host_id').tail()

host_id
4396       2
3867       2
3647       2
2881       2
1641537    1
Name: count, dtype: int64

In [186]:
df.query('host_id == 1641537')

,id,nome,host_id,host_name,bairro_group,bairro,latitude,longitude,room_type,price,minimo_noites,numero_de_reviews,ultima_review,reviews_por_mes,calculado_host_listings_count,disponibilidade_365,ultima_review_semestre,minimo_noites_categorico,disponibilidade_365_categorico,valor_preenchido
9520,7311209,SUPER HUGE ROOM IN GREENPOINT BK,1641537,Lauren,Brooklyn,Greenpoint,40.72503,-73.93909,Private room,53,7,7,2016-03-09 00:00:00,0.15,2,0,2016-1-semestre,Entre_3_a_7_Dias,0_Dias,False


Utilizei um loop para iterar sobre o DataFrame e identificar quais valores na coluna `calculado_host_listings_count` apresentavam inconsistências em relação aos valores esperados. 

Para isso, utilizei o tipo set, que retorna apenas os valores únicos de um conjunto de dados. Em cada iteração, apliquei a função len() ao conjunto de valores únicos, caso o resultado fosse diferente de 1, isso indicava a presença de inconsistências.

In [187]:
# Separei as colunas necessárias para essa análise, criei uma lista com os valores únicos 
# em "calculado_host_listings_count" e criei um dataframe vazio.
df_problema = df[['host_id', 'calculado_host_listings_count']]
ids = list(df_problema['calculado_host_listings_count'].unique())
df_teste = pd.DataFrame()

# loop com os valores únicos armazenando o resultado do tamanho do objeto set
for i in ids:
    tamanho_teste = len(set(df_problema.query(f'calculado_host_listings_count == {i}')\
                                       .value_counts('host_id')))
    
    # Compara se o valor é diferente de 1
    if tamanho_teste != 1:
        # Calcula a quantidade de host_ids verdadeira
        host_ids_counts_df = pd.DataFrame(df.query(f'calculado_host_listings_count == {i}')\
                                            .value_counts('host_id'))\
                                            .reset_index()
        
        # Filtra somente os valores incosistentes
        host_ids_counts = host_ids_counts_df[host_ids_counts_df['count'] != i]

        # Concatena os valores ao DataFrame. 
        df_teste = pd.concat([df_teste, host_ids_counts])

Parte deste erro é apresentado justamente devido a remoção de dados com preço "0", de todo modo, é necessária a correção, pois o host_id 2787 já está atribuido originalmente de forma incorreta.

In [188]:
# Dados incorretos
df_teste = df_teste.rename({'count': 'calculado_host_listings_count'}, axis = 1)
df_teste

,host_id,calculado_host_listings_count
3328,1641537,1
358,131697576,3
359,8993084,3
92,2787,5
93,101970559,4
94,86327101,3
168,15787004,4


In [189]:
# Loop para preencher os valores incorretos.
for i, row in df_teste.iterrows():
    df.loc[df['host_id'] == row['host_id'], 'calculado_host_listings_count'] = row['calculado_host_listings_count']

In [190]:
# Exibindo o resultado
df.query('host_id == 1641537')

,id,nome,host_id,host_name,bairro_group,bairro,latitude,longitude,room_type,price,minimo_noites,numero_de_reviews,ultima_review,reviews_por_mes,calculado_host_listings_count,disponibilidade_365,ultima_review_semestre,minimo_noites_categorico,disponibilidade_365_categorico,valor_preenchido
9520,7311209,SUPER HUGE ROOM IN GREENPOINT BK,1641537,Lauren,Brooklyn,Greenpoint,40.72503,-73.93909,Private room,53,7,7,2016-03-09 00:00:00,0.15,1,0,2016-1-semestre,Entre_3_a_7_Dias,0_Dias,False


- bairro_group - dummizar
- room_type - dummizar
- price - (y) normalizar
- numero_de_reviews - normalizar
- reviews_por_mes - normazliar
- calculado_host_listings_count - normalizar
- ultima_review_semestre - dumizar
- minimo_noites_categorico - dummizar
- disponibilidade_365_categorico - dummizar
- valor_preenchido - dummizar

- definir X e y

- separar em treino e teste

- criar classe para fazer tudo

In [191]:
df.bairro.nunique()

221

# Analise

In [193]:
df.head(2)

,id,nome,host_id,host_name,bairro_group,bairro,latitude,longitude,room_type,price,minimo_noites,numero_de_reviews,ultima_review,reviews_por_mes,calculado_host_listings_count,disponibilidade_365,ultima_review_semestre,minimo_noites_categorico,disponibilidade_365_categorico,valor_preenchido
0,2595,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,1,45,2019-05-21 00:00:00,0.38,2,355,2019-1-semestre,Entre_1_a_3_Dias,Entre_6_Meses_e_1_Ano,False
1,3647,THE VILLAGE OF HARLEM....NEW YORK !,4632,Elisabeth,Manhattan,Harlem,40.80902,-73.94190,Private room,150,3,0,anuncio_sem_review,0.00,1,365,anuncio_sem_review,Entre_1_a_3_Dias,1_Ano,True


In [194]:
df[['bairro_group', 
    'room_type', 
    'minimo_noites_categorico',
    'numero_de_reviews',
    'ultima_review_semestre',
    'reviews_por_mes',
    'calculado_host_listings_count',
    'disponibilidade_365_categorico',
    'valor_preenchido',
    'price'
]]

,bairro_group,room_type,minimo_noites_categorico,numero_de_reviews,ultima_review_semestre,reviews_por_mes,calculado_host_listings_count,disponibilidade_365_categorico,valor_preenchido,price
0,Manhattan,Entire home/apt,Entre_1_a_3_Dias,45,2019-1-semestre,0.38,2,Entre_6_Meses_e_1_Ano,False,225
1,Manhattan,Private room,Entre_1_a_3_Dias,0,anuncio_sem_review,0.00,1,1_Ano,True,150
2,Brooklyn,Entire home/apt,Entre_1_a_3_Dias,270,2019-1-semestre,4.64,1,Entre_6_Meses_e_1_Ano,False,89
3,Manhattan,Entire home/apt,Entre_1_e_2_Semanas,9,2018-2-semestre,0.10,1,0_Dias,False,80
4,Manhattan,Entire home/apt,Entre_1_a_3_Dias,74,2019-1-semestre,0.59,1,Entre_2_Meses_e_6_Meses,False,200
...,...,...,...,...,...,...,...,...,...,...
48889,Brooklyn,Private room,Entre_1_a_3_Dias,0,anuncio_sem_review,0.00,2,Entre_1 e 2 Semanas,True,70
48890,Brooklyn,Private room,Entre_3_a_7_Dias,0,anuncio_sem_review,0.00,2,Entre_1_Mes_e_2_Meses,True,40
48891,Manhattan,Entire home/apt,Entre_1_e_2_Semanas,0,anuncio_sem_review,0.00,1,Entre_2_Semanas_e_1_Mes,True,115
48892,Manhattan,Shared room,Entre_1_a_3_Dias,0,anuncio_sem_review,0.00,6,Entre_1_a_3_Dias,True,55
